# Imports

Inicialmente importa-se as bibliotecas que serão necessárias para rodar os códigos abaixo.

In [1]:
import os
import json
import urllib.request
import pandas as pd
import datetime
import calendar
import time

E é criada a estrutura de pastas que serão utilizadas para armazenar os arquivos que serão criados ao longo do processo. Em específico, a pasta *data* é a que armazenará a tabela com as informações.

In [2]:
%run '../codes/files/create_folders.py'
create_folders('', ['data'])

Directory "data" already exists...


# Série Histórica e Datas

A função abaixo retorna três parâmetros:
1. Tabela com uma série histórica, com uma data de início e uma data de término.
2. Data inicial em formato texto, para ser usada no nome do arquivo que vamos criar.
3. Data final em formato texto, para ser usada no nome do arquivo que vamos criar.

Caso não seja definida uma data final, será usado o dia de hoje.
Importante ter atenção a isso pois a SABESP disponibiliza atualizações dos dados as 9:00 (conforme site deles) e, caso o presente código seja rodado entre a meia noite e as 9:00, tentar-se-á obter informações não disponíveis, possivelmnete gerando erros. Para contornar isso, é possível inserir um parâmetro como *dia de hoje - 1*, por exemplo.

In [3]:
def get_data_tab(start='1970-1-1', end=None):
    """
    Function to create date table, with only on colum named 'Data' as index.
    With no 'end' parameter is passed, the function will return a table until today
    With no 'start' parameter is passed, the function will return a table staring in firts day of de 70's.
    
    The function return two more parameters to used in filenames:
    filename_start > first day of table 
    filename_end   > last day of table as string
    
    """
    
    # Dataframe to get dates
    if end is None:
        df = pd.DataFrame(pd.date_range(pd.to_datetime(start), end=datetime.date.today()), columns=['Data'])
        filename_start = str(datetime.datetime.strptime(str(start), '%Y-%m-%d').strftime('%Y-%m-%d'))
        filename_end   = str(datetime.datetime.strptime(str(datetime.date.today()), '%Y-%m-%d').strftime('%Y-%m-%d'))
    
    else:
        df = pd.DataFrame(pd.date_range(pd.to_datetime(start), end=pd.to_datetime(end)), columns=['Data'])
        filename_start = str(datetime.datetime.strptime(str(start), '%Y-%m-%d').strftime('%Y.%m.%d'))
        filename_end   = str(datetime.datetime.strptime(str(end), '%Y-%m-%d').strftime('%Y.%m.%d'))

    # Results
    df = df.set_index('Data')
    return df, filename_start, filename_end

In [4]:
start='2020-1-1'
df_day, filename_start, filename_end = get_data_tab(start=start)

In [5]:
#end='2020-3-28'
#end=str(datetime.date.today()-datetime.timedelta(days=1))
#df_day, filename_start, filename_end = get_data_tab(start=start, end=end)

# *Link* para fazer o *download* do Json

Inicialmente foram feitas tentativas diversas para melhor conhecimento do [*site* da SABESP](http://mananciais.sabesp.com.br/HistoricoSistemas?SistemaId=0) que disponibiliza as informações dos mananciais. Inicialmente tentou-se obter os dados pela técnica de *web scrapping*, até que descobriu-se que os dados são distribuídos por meio de uma API do SSD (Sistema de Suporte a Decisões).

Observou-se que a consulta manual apresenta os dados do mês da presente data até o primeiro dia do mês anterior. Por exemplo, se estamos no dia 25.**03**.2020, os dados apresentados serão dessa data até o dia 01.**02**.2020, retornando aproximadamente dados de 55 dias (30 dias de um mês hipotético e 25 do outro). O mesmo padrão irá ocorrer caso a consulta seja feita em 01.**03**.2020, a qual retornar-a os dados *até o primeiro dia do mês anterior*, ou seja, 01.**02**.2020.

Essa forma de "entregar" os dados foi considerada na requisição de dados pela API, a qual foi feita com uso do [urllib.request](https://stackoverflow.com/questions/32795460/loading-json-object-in-python-using-urllib-request-and-json-modules). A API tem seu *link* padrão apresentado abaixo, sendo inserido apenas duas variáveis: a data e o Sistema (aqui representado pelo 0 no final, que representa o Sistema Produtor Cantareira).

- http://mananciais.sabesp.com.br/api/Mananciais/RepresasSistemasNivel/2020-03-25/0

*ToDo*: O Presente código tem a finalidade de obter os dados unicamente do Sistema Cantareira e, portanto, não se pensou em adicionar a possibilidade de obter os dados dos outros sistemas produtores, por meio da inclusão e ajuste desse parâmetro definido e fixado como *0*.

In [6]:
site = ('http://mananciais.sabesp.com.br/api/Mananciais/RepresasSistemasNivel/' +
        str(filename_end.replace('.','-')) +
        '/0')
site

'http://mananciais.sabesp.com.br/api/Mananciais/RepresasSistemasNivel/2020-04-18/0'

Com o *link* da API com uma data definida, criou-se uma função para obter os dados em formato *json*, bastanto inserir o site.

In [10]:
def get_json(url):
    # Get Array with data
    webURL = urllib.request.urlopen(site)
    my_bytes = webURL.read()

    # Transform Array into JSON
    my_json = my_bytes.decode('utf8')
    data = json.loads(my_json)
    return json.dumps(data, indent=2, sort_keys=True)

In [11]:
jsn = get_json(site)
#print(jsn)

# Convertendo Json para tabela e extraíndo dados

Com o arquivo Json contendo todos os dados de mais de um mês atrás, segmentado em diversas chaves e subchaves, iniciou-se a segmentação do arquivo, convertendo o arquivo para uma tabela, com a qual tenho mais familiaridade para editar e filtrar.

A função abaixo tem essa aplicação e já aproveita para excluir duas colunas que, aparentemente, não agregam informações.

In [12]:
def json2table(jsn):
    # Create dataframe
    df = pd.read_json(jsn)

    # Delete columns
    return df.drop(['FlagHasError', 'Message'], axis=1)

In [13]:
df_json = json2table(jsn)
#df_json

## Sistema Produtor

Por meio do campo *SistemaId* é possível obter o código que define qual é o sistema produtor de água. Contudo, considerando que o presente *script* visa obter somente os dados do Sistema Cantareira, **tal função não será aplicada**.

In [14]:
df_json.loc['SistemaId']['ReturnObj']

0

## Data Final

Identifica data empregada na API, a partir da qual se obterá os dados *até o primeiro dia do mês anterior*. Não se vislumbra muita utilidade para essa informação nesse momento, tendo em vista que foi o usuário que definiu esse parâmetro na definição do *link* de acesso à API. Logo, **tal função não será aplicada**.

Cabe ressaltar que tal função é chamada em outras funções, para se obter a data do último registro da tabela, qu poderá, ou não, ser excluído.

In [15]:
def get_date(df):
    # JSON to dataframe    
    data = df_json.loc['Data']['ReturnObj']   
    
    # Results
    return pd.to_datetime(data, dayfirst=True).strftime('%Y-%m-%d')

In [16]:
#data = get_date(df_json)
#data

## Manobras Operacionais

Identifica todas as manobras listadas no site da SABESP. São dados mais descritivos, disponibilizados visando dar mais transparência a cadeia de comando para abrir ou fechar os reservatórios. Nesse primeiro momento tais dados não serão analisado e, portanto, **tal função não será aplicada**.

In [17]:
def get_manobras(df):
    # JSON to dataframe    
    lst = df.loc['ListaManobras']['ReturnObj']
    
    # Results
    return pd.json_normalize(lst)

In [18]:
#df_manobras = get_manobras(df_json)
#df_manobras

## Componentes do Sistema

### Reservatórios

A função abaixo lista os reservatórios (ou represas) do Sistema Cantareira e outras que estão integradas, de alguma maneira, ao Sistema, inserindo também o identificador de cada reservatório (*ComponenteId*).

Apesar de trata-se de uma tabela que não retorna dados temporais (por exemplo: vazão, volume e chuva), ou seja, que variam ao longo do tempo, é fundamental para rotular de qual reservatório que são os dados temporais que serão obtidos nas próximas funções, visto que eles se valem, majoritariamente, do campo *ComponenteId*.

In [19]:
def list_represas(df_json):
    # JSON to dataframe
    lst = df_json.loc['ListaRepresas']['ReturnObj']
    df = pd.json_normalize(lst)

    # Delete columns
    df = df.drop(['temChuva','temNivel', 'temQjus', 'temQnat', 'temVolume'], axis=1)

    # Results
    return df

In [20]:
tab_represas = list_represas(df_json)
tab_represas

,ComponenteId,Nome
0,1,Represa Jaguari/Jacareí
1,3,Represa Cachoeira
2,4,Represa Atibainha
3,5,Represa Paiva Castro
4,6,Represa Águas Claras
5,35,Represa Jaguari (Cesp)


### Estruturas (Túneis e outros Pontos de Medição)

A função abaixo lista os túneis e estações de monitoramento do Sistema Cantareira e outras que estão integradas, de alguma maneira, ao Sistema, inserindo também o identificador de cada local (*ComponenteId*).

Apesar de trata-se de uma tabela que não retorna dados temporais (por exemplo: vazão, volume e chuva), ou seja, que variam ao longo do tempo, contudo é fundamental para rotular de qual estrutura que são os dados temporais que serão obtidos nas próximas funções, visto que eles se valem, majoritariamente, do campo *ComponenteId* ou *abreviatura*.

In [21]:
def list_estruturas(df_json):
    # JSON to dataframe
    lst = df_json.loc['ListaLocais']['ReturnObj']
    df = pd.json_normalize(lst)

    # Delete columns
    df = df.drop(['Maximo','Minimo',
                  'Data','Dia',
                  'Valor','Unidade'],
                 axis=1)


    # Transform columns to list and reorder list
    col = df.columns.to_list()
    col.insert(0, col.pop(col.index('ComponenteId')))

    # Reindex Columns
    df = df.reindex(columns=col)

    # Results
    return df

In [22]:
tab_estruturas = list_estruturas(df_json)
tab_estruturas

,ComponenteId,Abreviatura,LocalMedicaoId,Nome,SistemaId
0,NaN,F-25bT,46,F-25bT,0
1,7.0,Q T5,10,Q Túnel 5,0
2,8.0,Q T6,9,Q Túnel 6,0
3,9.0,Q T7,7,Q Túnel 7,0
4,36.0,Q ESI,11,Q Elev. Santa Inês,0
5,48.0,Q PS-SC,47,Q Transf Paraíba do Sul - Atibainha (PS-SC),0
6,49.0,Q SC-PS,48,Q Transf Atibainha - Paraíba do Sul (SC-PS),0


## Dados Diários

Nessa seção que serão obtidos diversos dados relevantes na operação do Sistema Cantareira, tais como:
- Vazão natural em cada reservatório;
- Vazão afluente em cada reservatório;
- Vazão defluente em cada reservatório;
- Nível e Volume de cada reservatório;
- Dados de Precipitação de cada reservatório.

Inicialmente, definiu-se uma função para renomear *strings*, visto que estas constarão nos cabeçalhos das tabelas a serem criadas. Aplicou-se a função na *tab_represas* (criada acima) apenas para observar quais serão os nomes que constarão no cabeçalho das tabelas.

In [23]:
def rename_field(x):
    return(x.replace('/', '-').
           replace(' (', '-').
           replace(')', '').
           replace('Cesp', 'CESP').
           replace('Represa ', '').
           replace(' ', '')
          )

In [24]:
t = tab_represas
t['Nome_Header'] = t['Nome'].apply(lambda x: rename_field(x))
t

,ComponenteId,Nome,Nome_Header
0,1,Represa Jaguari/Jacareí,Jaguari-Jacareí
1,3,Represa Cachoeira,Cachoeira
2,4,Represa Atibainha,Atibainha
3,5,Represa Paiva Castro,PaivaCastro
4,6,Represa Águas Claras,ÁguasClaras
5,35,Represa Jaguari (Cesp),Jaguari-CESP


### Volume, QJusante e Chuva

Extraíndo os dados do json, por meio da chave *ListaDados* e subchave *Dados*, foi obtido os dados de volume, vazão defluente e precpitação de cada reservatório.

No arquivo json, as tabelas encontravam-se empilhadas (*flat table*), com uma coluna com o nome do reservatório. Portanto, foi necessário filtrar essas tabelas por reservatório, ajustar o cabeçalho inserindo o nome do reservatório em questão e, posteriromente, fazer um join das tabelas pelo campo *data*.

In [25]:
def list_volumes(df_json, keep_lastline=False):
    # JSON to dataframe
    lst = df_json.loc['ListaDados']['ReturnObj']
    df = pd.json_normalize(lst, 'Dados')

    # Define pivot to create new tables
    fields = df['Nome']
    fields = sorted(list(set(fields)))

    # Transform columns to list and reorder list
    col = df.columns.to_list()
    col.insert(0, col.pop(col.index('Data')))

    # Reindex Columns
    df = df.reindex(columns=col)

    # Create a blank table
    df_full,start,end = get_data_tab()

    for i in fields:
        # Define Nomes e Nomes de Tabelas
        j = rename_field(i)
        tab_name = 'tab_dados' + '_' + j
        
        # Filtra e cria das tabelas por fields (represas)
        locals()[tab_name] = df[df['Nome'] == i]

        # Deleta colunas
        locals()[tab_name] = locals()[tab_name].drop(['FlagConsolidado',
                                                      'NAMaxMax','NAMinMin',
                                                      'QJusanteMax','QJusanteMin',
                                                      'NivelUltimoDia',
                                                      'SistemaId','ComponenteId',
                                                      'UltimoDia',
                                                      'VazaoJusantePrincipal','VazaoJusanteSecundaria',
                                                      'VolumeOperacionalUltimoDia','VolumePorcentagemUltimoDia',
                                                      'VolumeTotalUltimoDia','Nome'], axis=1)

        # Renomeia as colunas
        locals()[tab_name].columns = [x if x=='Data' else j+'_'+x for x in locals()[tab_name].columns]

        # Convert Data Column (object) to datatime colum
        locals()[tab_name]['Data'] = pd.to_datetime(locals()[tab_name]['Data'])

        # Merge all tables
        df_full = pd.merge(df_full,locals()[tab_name],on='Data',how='left')

    # Results
    df_full = df_full.set_index('Data')
    df_full.dropna(how='all', inplace=True)
    
    if not(keep_lastline):
        data = get_date(df_json)
        df_full.drop(pd.Timestamp(data), axis=0, inplace=True)
    else:
        pass
    
    return df_full

In [26]:
tab_volumes = list_volumes(df_json, False)
tab_volumes.head(5)

KeyError: 'Data'

### Vazão Afluente e Vazão Natural

Extraíndo os dados do json, por meio da chave *ListaDados* e subchave *Qnat*, foram obtido os dados de vazão afluente e vazão naturalde cada reservatório.

No arquivo json, as tabelas encontravam-se empilhadas (*flat table*), com uma coluna com o nome do reservatório. Portanto, foi necessário filtrar essas tabelas por reservatório, ajustar o cabeçalho inserindo o nome do reservatório em questão e, posteriromente, fazer um join das tabelas pelo campo *data*.

In [ ]:
def list_vazao(df_json, keep_lastline=False):
    # JSON to dataframe
    lst = df_json.loc['ListaDados']['ReturnObj']
    df = pd.json_normalize(lst, 'Qnat')

    # Merge Tables
    df = pd.merge(df, tab_represas, on='ComponenteId', how='outer')

    # Define pivot to create new tables
    fields = df['Nome']
    fields = sorted(list(set(fields)))
    
    # Transform columns to list and reorder list
    col = df.columns.to_list()
    col.insert(0, col.pop(col.index('Data')))

    # Reindex Columns
    df = df.reindex(columns=col)

    # Create a blank table
    df_full,start,end = get_data_tab()

    for i in fields:
        # Define Nomes e Nomes de Tabelas
        j = rename_field(i)
        tab_name = 'tab_vazaonatural'+'_'+j
        
        # Filtra e cria das tabelas por fields (represas)
        locals()[tab_name] = df[df['Nome'] == i]

        # Deleta colunas
        locals()[tab_name] = locals()[tab_name].drop(['ComponenteId','Nome',
                                                      'VazaoAfluenteMax','VazaoAfluenteMin',
                                                      'VazaoNaturalMax','VazaoNaturalMin'],axis=1)

        # Renomeia as colunas
        locals()[tab_name].columns = [x if x=='Data' else j+'_'+x for x in locals()[tab_name].columns]

        # Convert Data Column (object) to datatime colum
        locals()[tab_name]['Data'] = pd.to_datetime(locals()[tab_name]['Data'])

        # Merge all tables
        df_full = pd.merge(df_full,locals()[tab_name], on='Data', how='left')

    # Results
    df_full = df_full.set_index('Data')
    df_full.dropna(how='all', inplace=True)
    
    if not(keep_lastline):
        data = get_date(df_json)
        df_full.drop(pd.Timestamp(data), axis=0, inplace=True)
    else:
        pass
    
    return df_full

In [ ]:
tab_vazao = list_vazao(df_json, False)
tab_vazao.head(5)

### Sistema Equivalente

Extraíndo os dados do json, por meio da chave *ListaDados*, foram obtido os dados do Sistema Equivalente.

In [ ]:
def list_SE(df_json, keep_lastline=False):
    # JSON to dataframe
    lst = df_json.loc['ListaDados']['ReturnObj']
    df = pd.json_normalize(lst)

    # Delete columns
    df = df.drop(['Dados','Data','Qnat'], axis=1)

    # Transform columns to list and reorder list
    col = df.columns.to_list()

    # Functions to rename
    col = ['SE_'+x for x in col]
    col = [x.replace('SistemaEquivalente.', '').replace('SE_Data', 'Data') for x in col]

    # Rename Columns
    df.columns = col

    # Convert Data Column (object) to datatime colum
    df['Data'] = pd.to_datetime(df['Data'])

    # Results
    df = df.set_index('Data')
    df.dropna(how='all', inplace=True)
    
    if not(keep_lastline):
        data = get_date(df_json)
        df.drop(pd.Timestamp(data), axis=0, inplace=True)
    else:
        pass
    
    return df

In [ ]:
tab_SE = list_SE(df_json, False)
tab_SE.head(5)

### Vazão dos Túneis e outros Pontos de Medição

Extraíndo os dados do json, por meio da chave *ListaDadosLocais* e subchave *Dados*, foram obtido os dados de vazão dos túneis Q7, Q6 Q5 e outros.

No arquivo json, as tabelas encontravam-se empilhadas (*flat table*), com uma coluna com o nome da estrutura. Portanto, foi necessário filtrar essas tabelas por estrutura, adicionando ao cabeçalho seu repectivo nome e, posteriormente, fazer um join das tabelas pelo campo *data*.

In [ ]:
def list_vazaoestruturas(df_json, keep_lastline=False):
    # JSON to dataframe
    lst = df_json.loc['ListaDadosLocais']['ReturnObj']
    df = pd.json_normalize(lst, 'Dados')

    # Define pivot to create new tables
    fields = df['Abreviatura']
    fields = sorted(list(set(fields)))

    # Transform columns to list and reorder list
    col = df.columns.to_list()
    col.insert(0, col.pop(col.index('Data')))
    col.append(col.pop(col.index('Unidade')))

    # Reindex Columns
    df = df.reindex(columns=col)

    # Create a blank table
    df_full,start,end = get_data_tab()

    for i in fields:
        # Define Nomes e Nomes de Tabelas
        j = rename_field(i)
        tab_name = 'tab_dados' + '_' + j
        
        # Filtra e cria das tabelas por fields (represas)
        locals()[tab_name] = df[df['Abreviatura'] == i]

        # Deleta colunas
        locals()[tab_name] = locals()[tab_name].drop(['Maximo', 'Minimo', 'Dia', 'Abreviatura', 'ComponenteId', 'LocalMedicaoId', 'Nome', 'SistemaId'],axis=1)

        # Renomeia as colunas
        locals()[tab_name].columns = [x if x=='Data' else j+'_'+x for x in locals()[tab_name].columns]

        # Convert Data Column (object) to datatime colum
        locals()[tab_name]['Data'] = pd.to_datetime(locals()[tab_name]['Data'])

        # Merge all tables
        df_full = pd.merge(df_full,locals()[tab_name],on='Data',how='left')

    # Results
    df_full = df_full.set_index('Data')
    df_full.dropna(how='all', inplace=True)
    
    if not(keep_lastline):
        data = get_date(df_json)
        df_full.drop(pd.Timestamp(data), axis=0, inplace=True)
    else:
        pass
    
    return df_full

In [ ]:
tab_vazaoestruturas = list_vazaoestruturas(df_json, False)
tab_vazaoestruturas.head(5)

## Dados Horários

Referem-se a transposição da bacia do rio Paraíba do Sul para a bacia do rio Piracicaba, por meio da Estação Elevatória de Água Bruta (EEAB) Jaguari, que despeja água na represa Atibainha.

Tais dados não serão aqui considerados, visto que já se encontram discretizados em dado diário na tabela acima. Logo, **tal função não será aplicada**.

### Estrutura de Transposição

In [ ]:
def list_EEAB(df_json):
    # JSON to dataframe
    lst = df_json.loc['ListaEspecial']['ReturnObj']
    df = pd.json_normalize(lst)
    
    # Results
    return df

In [ ]:
#tab_vazaoEEAB_pontos = list_EEAB(df_json)
#tab_vazaoEEAB_pontos

### Vazão de Transposição

Dados horários, transferência 

In [ ]:
def list_vazao_EEAB(df_json):
    # JSON to dataframe
    lst = df_json.loc['ListaDadosEspecial']['ReturnObj']
    df = pd.json_normalize(lst, 'Dados')
    
    # Results
    return df

In [ ]:
#tab_vazaoEEAB = list_vazao_EEAB(df_json)
#tab_vazaoEEAB

# Resultado: Série Histórica do Sistema Cantareira

Com todas as funções definidas, é possível aplicar tais funções sequencialmente, visando criar uma série histórica com todos os dados do Sistema Cantareira.

Sabendo que a API obtem os dados *até o primeiro dia do mês anterior*, criou-se uma função que monta uma lista de todos os meses e anos a partir de uma data *start*. Posterirmente define-se o dia seguinte ao último dia do mês, ou seja:
1. Último dia de um determinado mês [último dia].[mês].[ano];
2. Dia seguinte, ou seja, virada de mês, [último dia].[mês].[ano] + 1, que resultará, obrigatoriamente, em [primeiro dia do mês].[mês+1].[ano];

Com isso são obtidos do mês anterior e excluído o último dia já do mês subsquente. Como resultado, para cada iteração, teremos o conjunto de dados de um único mês, os quais serão apensados a cada iteração.

## Série Histórica e Datas

In [ ]:
# Table with start date
start='1990-1-1'
df_day, filename_start, filename_end = get_data_tab(start)

In [ ]:
# Table with start and end date
#end='2000-1-1'
#end=datetime.date.today()
#end=str(datetime.date.today()-datetime.timedelta(days=1))
#df_day, filename_start, filename_end = get_data_tab(start=start, end=end)

In [ ]:
# Years's List
list_year = df_day.index.year
list_year = list(set(list_year))
list_year = sorted(list_year, reverse = True)

# Month's List
list_month = df_day.index.month
list_month = list(set(list_month))
list_month = sorted(list_month, reverse = True)

## *Loop*

In [ ]:
# Zera os Objetos
tabs_volumes         = []
tabs_vazao           = []
tabs_SE              = []
tabs_vazaoestruturas = []

# Function to loop
for y in list_year:
    for m in list_month:
        # Variáveis de Data
        lastday    = calendar.monthrange(y, m)[1]
        date       = datetime.date(y, m, lastday)
        date_site  = datetime.date(y, m, lastday)+datetime.timedelta(days=1)
        today      = datetime.date.today()
        
        if date_site < today:
            if date_site.month == today.month and date_site.year == today.year:
                day = today
                keep_lastline=True
                print(str(datetime.datetime.now().time())+' | '+str(day) + ' > Dados de hoje até inicio do mês anterior > keep_lastline='+str(keep_lastline))
        
            elif date_site.month != today.month or date_site.year != today.year:
                day = date_site
                keep_lastline=False
                print(str(datetime.datetime.now().time())+' | '+str(day) + ' > Dados do mês anterior > keep_lastline='+str(keep_lastline))

                # Site
                site = 'http://mananciais.sabesp.com.br/api/Mananciais/RepresasSistemasNivel/' + str(day) + '/0'

            # Functions
            jsn = get_json(site)
            time.sleep(30)
            df_json = json2table(jsn)
            #data = get_date(df_json)
            tab_manobras = get_manobras(df_json)
            tab_represas = list_represas(df_json)

            # Dados
            tab_volumes         = list_volumes(df_json, keep_lastline)
            tab_vazao           = list_vazao(df_json, keep_lastline)
            tab_SE              = list_SE(df_json, keep_lastline)
            tab_vazaoestruturas = list_vazaoestruturas(df_json, keep_lastline)

            # Concat Data
            tabs_volumes.append(tab_volumes)
            tabs_vazao.append(tab_vazao)
            tabs_SE.append(tab_SE)
            tabs_vazaoestruturas.append(tab_vazaoestruturas)

        elif date_site > today:
            day = date_site
            print(str(day) + ' > Data que não atende condições')
            
        else:
            print('Erro')
            
print('Fim!')

## Conctatena e une tabelas

In [ ]:
# Concat Data
tabs_volumes         = pd.concat(tabs_volumes)
tabs_vazao           = pd.concat(tabs_vazao)
tabs_SE              = pd.concat(tabs_SE)
tabs_vazaoestruturas = pd.concat(tabs_vazaoestruturas)

In [ ]:
# One Table
tabs_volumes = pd.concat([tabs_volumes, tabs_vazao], axis=1)
tabs_volumes = pd.concat([tabs_volumes, tabs_SE], axis=1)
tabs_volumes = pd.concat([tabs_volumes, tabs_vazaoestruturas], axis=1)

# Merge
df_final = pd.merge(df_day, tabs_volumes, left_index=True, right_index=True, how='left')

## Export to CSV

In [ ]:
# Export
df_final.dropna(how='all', inplace=True)

filename='tab_Cantareira_'+filename_end+'_até_'+filename_start+'.csv'
df_final.to_csv(os.path.join('data', filename),
                     index=True,
                     header=True,
                     sep=';',
                     date_format='%d/%m/%Y',
                     decimal=',')